preproceso de datos para entrenar la red

## Datos

 En este caso no hay dataset etiquetado de faithful, por tanto crearé en datset yo descargando los datos de la página oficial del pack de texturas

In [6]:
import os
import pandas as pd

def build_dataset_from_faithfull(folder_path, image_exts={'.png'}):
    #Lee imágenes de la carpeta 'items_VERSION_TAMxTAM/' en datos y crea el dataframe
    records = []

    for root, _, files in os.walk(folder_path):
        for file in files:
            _, ext = os.path.splitext(file)
            if ext.lower() not in image_exts:
                continue

            file_path = os.path.join(root, file)
            with open(file_path, 'rb') as f:
                data = f.read()

            label = os.path.splitext(file)[0].replace('_', ' ') # creamos las etiquetas desde el nombre del archivo
            records.append({
                'image': {'bytes': data},
                'label': label
            })

    df = pd.DataFrame(records)
    return df


Version 1.21.1 faithful32x32

In [ ]:
version="1.21.1"
tam=32

items_folder = f"datos/item_{version}_{str(tam)}x{str(tam)}"
df = build_dataset_from_faithfull(items_folder)


In [8]:
df.head()

,image,label
0,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia boat
1,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia chest boat
2,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia door
3,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia hanging sign
4,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia sign


In [9]:
df.shape

(604, 2)

#### Guardar los datos en un parquet

In [ ]:
df.to_parquet(f"datos/faithful_{version}_{str(tam)}.parquet")